#구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#import

In [ ]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import torch.optim as optim
from torch.nn import init
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Utils

In [ ]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

#Csutom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False, trg= False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.trg = trg

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      if self.trg:
        img_path = self.data.iloc[idx, 1]
        path_len = len(img_path)
        image = cv2.imread('/content/drive/MyDrive/dacon/san'+img_path[1:path_len])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
      else :
        img_path = self.data.iloc[idx, 1]
        path_len = len(img_path)
        image = cv2.imread('/content/drive/MyDrive/dacon/san'+img_path[1:path_len])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_path = self.data.iloc[idx, 2]
        mask_path_len = len(mask_path)
        mask = cv2.imread('/content/drive/MyDrive/dacon/san'+ mask_path[1:mask_path_len], cv2.IMREAD_GRAYSCALE)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

#Data Loader

In [ ]:
transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

train_dataset = CustomDataset(csv_file='/content/drive/MyDrive/dacon/san/train_source.csv', transform=transform)
train_target_dataset = CustomDataset(csv_file='/content/drive/MyDrive/dacon/san/train_target.csv', transform=transform,trg=True)
val_dataset = CustomDataset(csv_file='/content/drive/MyDrive/dacon/san/val_source.csv', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
train_target_dataloader = DataLoader(train_target_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

#Define Model

#Model Train

#Inference

In [ ]:
test_dataset = CustomDataset(csv_file='/content/drive/MyDrive/dacon/san/test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        outputs = model(images)
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        # batch에 존재하는 각 이미지에 대해서 반복
        for pred in outputs:
            pred = pred.astype(np.uint8)
            pred = Image.fromarray(pred) # 이미지로 변환
            pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
            pred = np.array(pred) # 다시 수치로 변환
            # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
            for class_id in range(12):
                class_mask = (pred == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else: # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

100%|██████████| 119/119 [10:12<00:00,  5.15s/it]


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/dacon/san/sample_submission.csv')
submit['mask_rle'] = result
submit

,id,mask_rle
0,TEST_0000_class_0,400981 4 401941 4 402892 22 403852 22 404812 2...
1,TEST_0000_class_1,440637 4 441597 4 442557 4 443517 4 444477 4 4...
2,TEST_0000_class_2,1 111 610 17 657 415 1570 17 1617 420 2521 476...
3,TEST_0000_class_3,-1
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,162794 4 163754 4 167590 8 168550 8 169510 12 ...
22772,TEST_1897_class_8,104 531 682 116 1064 531 1642 116 2024 531 260...
22773,TEST_1897_class_9,204198 4 205158 4 206118 4 207078 4 208038 4
22774,TEST_1897_class_10,-1


In [ ]:
submit.to_csv('/content/drive/MyDrive/dacon/san/unet3p_submit.csv', index=False)